In [10]:
from modules import DATA_PROCESSING, METHODS
import pandas as pd

dp = DATA_PROCESSING("../data_share.csv")
data = pd.DataFrame(dp.mdmi())

In [11]:
from sklearn.model_selection import train_test_split

PC = METHODS()

def run(dataset, model_name, kernel, x):
    features = dataset.iloc[:, 1:]
    labels = dataset.iloc[:, 0]
    # print(labels.shape, Counter(labels))

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
    # print("Test size: ", y_test.shape)

    X_train, X_test, scaler = PC.scaling(X_train, X_test, x)

    algorithm = model_name(kernel=kernel, n_restarts_optimizer=5)
    algorithm.fit(X_train, y_train)

    y_pred = algorithm.predict(X_test)
    return y_test, y_pred, scaler

In [12]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C, RationalQuadratic as RQ
from sklearn.metrics import r2_score
import numpy as np
import math

model = GaussianProcessRegressor
kernel = C()*RQ(length_scale=50, alpha=1)
times = 20

best_score = -math.inf
best_scaler = ""

for i in range(4):
    container = []
    for j in range(times):
        test, pred, scaler = run(data, model, kernel, i)
        score = r2_score(test, pred)
        container.append(score)

    new_score = np.mean(container)
    if new_score > best_score:
        best_score = new_score
        best_scaler = scaler

print("Best Scaler: ", best_scaler)
print("Score: ", best_score)

Best Scaler:  MinMaxScaler()
Score:  0.7594511533958279
